# Visualise difference cases of lighthouse illusion (from python script)

The visualisation by VPython library uses browser window. In notebook only the static graphics can be shown by the canvas and hence animation cannot be shown directly - I haven't found any solution to this. Please let me know if you know how to tackle this, though not of importance as we can see it by executing the script directly.\

**Therefore for now please run the python script `LighthouseSimulation.py` directly to see the simulation.** \
Please change the `distance` variable to modify the distance between the person and the lighthouse.
- `distance` = 20 (very close to the lighthouse), no illusion. An interesting finding is that, even at this short distance, the rotating light beam seems move horizontally already. (meaning the effects of perspective, or the prior from our explanation approach, are significant)
- `distance` = 100, weak illusion. There are some indications that the beam comes from the front, but we can still probably realise it is from the back as the convergence is not significant.
- `distance` = 350, strong illusion. The feeling of convergence in the front is strong. It may be more obvious if there are more noises in the distant end. 

I believe the simulation matches our expectation =)

For the graphic visualisation:
- Scroll up / down (for trackpad: pinch two fingers open / closed): **zoom in / out**
    - please note that the rate of zoom in / out through pinching is very fast, and the scene will quickly disappear - If so, please re-run the cell.
- Press the right click and drag: **rotate the view angle**

# Prior Simulation & Visualisation

In [ ]:
pip -install vpython

In [1]:
from vpython import *

<IPython.core.display.Javascript object>

`PriorSimulation.py` generates the simulated prior **before projection**, with graphic visualisation. \
The simulated prior is represented by four 2d numpy arrays, \
- surfaces_left_top
- surfaces_left_bottom
- surfaces_right_top
- surfaces_right_bottom 

each of which contains the **(x,y) coordinate** and the **distance** of the respective corner of the simulated surface \

For the graphic visualisation:
- Scroll up / down (for trackpad: pinch two fingers open / closed): **zoom in / out**
    - please note that the rate of zoom in / out through pinching is very fast, and the scene will quickly disappear - If so, please re-run the cell.
- Press the right click and drag: **rotate the view angle**

In [4]:
canvas()
%run vPython/PriorSimulation.py

<IPython.core.display.Javascript object>

# Project the 3D prior to 2D through pinhole camera

# Visualise the prior distribution through difference angles

In [5]:
def homogenise(points: np.array):
    """Converts non-homogeneous 2D or 3D coordinates into homogeneous coordinates.
    For example
    [x1 x2 x3]    [x1 x2 x3]
    [y1 y2 y3] -> [y1 y2 y3]
                  [1  1  1]
    Parameters
    ----------
    points : numpy.array, shape (dimensions, nr_points)
    Returns
    --------
    points : numpy.array, shape (dimensions+1, nr_points)
    """

    return np.concatenate((points, np.ones((1, points.shape[1]))), axis=0)

def forwardprojectP(points: np.array, P: np.array, image_size, image=None):
    """
    # Project 3D points, defined by [X Y Z]', onto an image plane defined by a 3x4 projection matrix P.
    # Returns those 3D points that are withing the FOV of the camera (i.e. filters out those points
      that are outside of the FOV), the corresponding uv-image coordinates, and a depth map.
    
    # Additionally, if an image is given, RGB for each 3D point is returned - None 
    
    Parameters
    ----------
    points : numpy.array, shape (3, nr_points)
    P : numpy array, shape (3, 4), Camera projection matrix P = K[R | t]
        In our case, as camera is positioned in the center, 
        [R | t] = [1, 0, 0, 0]
                  [0, 1, 0, 0]
                  [0, 0, 1, 0]
        K = [f, 0, px ]    [0.01,    0, 216]
            [0, f, py ] => [   0, 0.01, 216] 
            [0, 0,  1 ]    [   0,    0,   1]
                  
    image_size : tuple
            Image size, (512, 512)
    image : numpy.array, optional
            Image used for defining colors for each point (default is None).
    
    Returns
    -------
    Our Case:
    (3D points, uv-coordinates, depth map) : numpy array
        If no image is given (i.e. is None). Shapes are (3, nr_points), (3, nr_points) and (rows, cols)
    (3D points, uv-coordinates, RGB, depth map) : numpy array
        If image is given. Shapes are (3, nr_points), (3, nr_points), (3, nr_points) and (rows, cols)
    """

    # Convert the image_size into a tuple. It might already be a tuple, but let's just make sure
    image_size = (int(image_size[0]), int(image_size[1]))
    depth_map = np.ones(image_size) * np.nan

    try:
        # Convert points into homogeneous form
        points = homogenise(points)

        # Project points to image
        uv = np.matmul(P, points)

        # Filter points that fall behind the camera
        mask = uv[2, :] < 0.0
        uv = uv[:, ~mask]
        points = points[:, ~mask]

        # Normalize coordinates
        uv[0, :] /= uv[2, :]
        uv[1, :] /= uv[2, :]
        uv[2, :] /= uv[2, :]

        # Mask out points that don't fall withing the given image (i.e. are outside of FOV)
        mask = (uv[0, :] < 0) | (uv[0, :] > (image_size[1] - 1)) | (uv[1, :] < 0) | (uv[1, :] > (image_size[0] - 1))
        points = points[:, ~mask]
        uv = uv[:, ~mask]
        
#         print(np.round(uv[1, :]).astype(int))
        
        # Generate a depth map
        depth_map[np.round(uv[1, :]).astype(int), np.round(uv[0, :]).astype(int)] = points[2, :]

        # Handle colors, if given
        if image is None:
            return points[:3, :], uv, depth_map
        else:
            RGB = image[np.round(uv[1, :]).astype(int), np.round(uv[0, :]).astype(int), :]
            return points[:3, :], uv, RGB, depth_map
    except Exception as e:
        raise

In [6]:
ls

Illusion in Bayesian.ipynb      depth.npy
LICENSE                         depth_0.npy
README.md                       open3D/
Simulation with Graphics.ipynb  vPython/
Snapshots/


In [11]:
%run open3D/OnePriorSimulation.py

RuntimeError: [1;31m[Open3D Error] (open3d::visualization::rendering::EngineInstance::EngineInstance()) /Users/runner/work/Open3D/Open3D/cpp/open3d/visualization/rendering/filament/FilamentEngine.cpp:123: EGL Headless is not supported on this platform.
[0;m